Решить задачу просрочки кредита используя для прогназирования нейронные сети. Посмотреть работу с несбалансированными данными. 


Набор данных представляет собой исторические данные по 251503 заемщикам (https://www.kaggle.com/c/GiveMeSomeCredit/data ).
Задача является бинарной классификацией. Цель - предсказать будет ли тот или иной заемщик испытывать финансовые трудности в ближайшие 2 года, т.е. будет ли просрочка по займу. 
Выборка разделена на тренировочную и тестовую ( 150000 в тренировочной части, 101503 в тестовой).

In [1]:
!pip install -r requirements.txt

In [2]:
import pandas as pd
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

2024-03-17 22:30:21.172204: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-17 22:30:21.174113: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-17 22:30:21.204847: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 22:30:21.204880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 22:30:21.207247: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Импорт всех инструментов

In [3]:
df = pd.read_csv('cs-training.csv')
df.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
df.corr()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
Unnamed: 0,1.000000,0.002801,0.002372,0.004403,-0.000571,-0.002906,0.002632,0.004586,-0.001104,-0.000666,-0.000777,-0.000055
SeriousDlqin2yrs,0.002801,1.000000,-0.001802,-0.115386,0.125587,-0.007602,-0.019746,-0.029669,0.117175,-0.007038,0.102261,0.046048
RevolvingUtilizationOfUnsecuredLines,0.002372,-0.001802,1.000000,-0.005898,-0.001314,0.003961,0.007124,-0.011281,-0.001061,0.006235,-0.001048,0.001557
age,0.004403,-0.115386,-0.005898,1.000000,-0.062995,0.024188,0.037717,0.147705,-0.061005,0.033150,-0.057159,-0.213303
NumberOfTime30-59DaysPastDueNotWorse,-0.000571,0.125587,-0.001314,-0.062995,1.000000,-0.006542,-0.010217,-0.055312,0.983603,-0.030565,0.987005,-0.002680
DebtRatio,-0.002906,-0.007602,0.003961,0.024188,-0.006542,1.000000,-0.028712,0.049565,-0.008320,0.120046,-0.007533,-0.040673
MonthlyIncome,0.002632,-0.019746,0.007124,0.037717,-0.010217,-0.028712,1.000000,0.091455,-0.012743,0.124959,-0.011116,0.062647
NumberOfOpenCreditLinesAndLoans,0.004586,-0.029669,-0.011281,0.147705,-0.055312,0.049565,0.091455,1.000000,-0.079984,0.433959,-0.071077,0.065322
NumberOfTimes90DaysLate,-0.001104,0.117175,-0.001061,-0.061005,0.983603,-0.008320,-0.012743,-0.079984,1.000000,-0.045205,0.992796,-0.010176
NumberRealEstateLoansOrLines,-0.000666,-0.007038,0.006235,0.033150,-0.030565,0.120046,0.124959,0.433959,-0.045205,1.000000,-0.039722,0.124684


In [5]:
df.shape

(150000, 12)

In [6]:
df_dropped_null = df.dropna(inplace=False)

In [7]:
df_dropped_null.shape

(120269, 12)

Видим, что если просто удалить пустые значения, то мы потеряем много данных, нужно их заполнить чем то

In [8]:
df.head(10)

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
5,6,0,0.213179,74,0,0.375607,3500.0,3,0,1,0,1.0
6,7,0,0.305682,57,0,5710.000000,NaN,8,0,3,0,0.0
7,8,0,0.754464,39,0,0.209940,3500.0,8,0,0,0,0.0
8,9,0,0.116951,27,0,46.000000,NaN,2,0,0,0,NaN
9,10,0,0.189169,57,0,0.606291,23684.0,9,0,4,0,2.0


In [9]:
df['Unnamed: 0'].shape

(150000,)

In [10]:
df['Unnamed: 0'].fillna(0, inplace=True)

/tmp/ipykernel_29654/3944213137.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Unnamed: 0'].fillna(0, inplace=True)


In [11]:
df['Unnamed: 0'].dropna().shape

(150000,)

In [12]:
df['age'].dropna().shape # нет ячеек без возраста

(150000,)

In [13]:
df['SeriousDlqin2yrs'].fillna(0, inplace=True)

/tmp/ipykernel_29654/455679639.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['SeriousDlqin2yrs'].fillna(0, inplace=True)


In [14]:
df['RevolvingUtilizationOfUnsecuredLines'].isnull().any() # видим что здесь нет пустых ячеек тоже

False

In [15]:
df['NumberOfTime30-59DaysPastDueNotWorse'].isnull().any()

False

In [16]:
df['DebtRatio'].isnull().any() # тоже нет

False

In [17]:
df['MonthlyIncome'].isnull().any() # есть! Значит заполним их средним

True

In [18]:
meanMI = df['MonthlyIncome'].mean
df['MonthlyIncome'].fillna(meanMI, inplace=True)

/tmp/ipykernel_29654/1144724241.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<bound method Series.mean of 0          9120.0
1          2600.0
2          3042.0
3          3300.0
4         63588.0
           ...   
149995     2100.0
149996     5584.0
149997        NaN
149998     5716.0
149999     8158.0
Name: MonthlyIncome, Length: 150000, dtype: float64>' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['MonthlyIncome'].fillna(meanMI, inplace=True)


In [19]:
df['NumberOfOpenCreditLinesAndLoans'].isnull().any()

False

In [20]:
df['NumberOfTimes90DaysLate'].isnull().any()

False

In [21]:
df.dropna().shape 

(146076, 12)

Чтобы не повторять эти действия кучу раз, я проверил как много еще осталось пустых значений, и увидел что их мало. Думаю их можно просто отбросить

In [22]:
df.dropna(inplace=True)

In [23]:
X = df.drop(columns='SeriousDlqin2yrs')
y = df['SeriousDlqin2yrs']

In [24]:
df.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [25]:
y = to_categorical(y) # one-hot encoding

In [26]:
X['MonthlyIncome'] = pd.to_numeric(df['MonthlyIncome'], errors='coerce')

Для лучшего обучения модели скалируем их

In [27]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

Пора приступить к созданию модели

In [28]:
import tensorflow as tf
from keras.layers import Dense, Dropout, ReLU
from keras.models import Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tqdm import tqdm

In [29]:
model = Sequential()
model.add(Dense(64, input_shape=(X_scaled.shape[1],)))
model.add(ReLU())
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(ReLU())
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

In [30]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
model.fit(X, y, epochs=10, batch_size=64)

Epoch 1/10
2283/2283 [==============================] - 3s 893us/step - loss: 0.9387 - accuracy: 0.9325
Epoch 2/10
2283/2283 [==============================] - 2s 958us/step - loss: 0.2520 - accuracy: 0.9326
Epoch 3/10
2283/2283 [==============================] - 2s 1ms/step - loss: 0.2498 - accuracy: 0.9326
Epoch 4/10
2283/2283 [==============================] - 2s 1ms/step - loss: 0.2490 - accuracy: 0.9326
Epoch 5/10
2283/2283 [==============================] - 2s 1ms/step - loss: 0.2487 - accuracy: 0.9326
Epoch 6/10
2283/2283 [==============================] - 2s 915us/step - loss: 0.2480 - accuracy: 0.9326
Epoch 7/10
2283/2283 [==============================] - 2s 844us/step - loss: 0.2480 - accuracy: 0.9326
Epoch 8/10
2283/2283 [==============================] - 2s 837us/step - loss: 0.2475 - accuracy: 0.9326
Epoch 9/10
2283/2283 [==============================] - 2s 886us/step - loss: 0.2476 - accuracy: 0.9326
Epoch 10/10
2283/2283 [==============================] - 2s 991us/step

In [32]:
batch_size=64
epochs=5
step_per_epoch = len(X_scaled) // batch_size
cross_entropy = BinaryCrossentropy(from_logits=True)

Сделать обучение без метода фит не успел, я попытался, но обучение не идет

Не запускать

In [33]:
# for epoch in range(epochs):
#     print(f"Эпоха номер {epoch+1}")
#     progress_bar = tqdm(range(step_per_epoch))
#     for step in progress_bar:
#         batch_x = X_scaled[step*batch_size:(step+1)*batch_size]
#         # print(batch_x[0])
#         batch_y = y[step*batch_size:(step+1)*batch_size]
#         # print(batch_y[0])
#         with tf.GradientTape() as tape:
#             y_pred = model(batch_x, training=True)
#             loss = cross_entropy(batch_y, y_pred)
#         gradients = tape.gradient(loss, model.trainable_variables)
#         optimizer.apply_gradients(zip(gradients, model.trainable_variables))
#         accuracy = tf.keras.metrics.binary_accuracy(batch_y, y_pred)

#         progress_bar.set_description(f'Шаг {step+1}/{step_per_epoch}, Loss: {loss.numpy()}, accuracy: {accuracy.numpy()}')


Сохраним модель

In [42]:
import tf2onnx

In [51]:
onnx_model = tf2onnx.convert.from_keras(model, output_path='model.onnx')

2024-03-17 22:39:03.911477: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-03-17 22:39:03.911591: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-03-17 22:39:03.956600: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-03-17 22:39:03.956692: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
